# Gemini Basic

### Text generation

In [1]:
from google import genai

client = genai.Client()  # Automatically get the API key from .env

'''
The free tier in Gemini API allows for (Gemini 2.5 flash):
- Requests per minute (RPM): 10
- Tokens per minute (TPM): 250,000
- Requests per day (RPD): 250
'''

reponse = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="Generate 5 jokes about parenting in markdown format."
)

print(reponse.text)

Here are 5 jokes about parenting:

1.  Why do parents always look tired?
    Because "sleep like a baby" is a cruel, cruel lie.

2.  My kids have selective hearing. They can hear a candy wrapper crinkle from two rooms away, but not me asking them to clean their room.

3.  What's it like having a toddler?
    Imagine living with a tiny, drunk roommate who thinks glitter is a condiment and the floor is a canvas.

4.  My child just discovered the word "why." Our conversations now resemble a philosophical debate between a zen master and a tiny, incredibly persistent parrot.

5.  Dinner with kids is basically a culinary hostage negotiation where the terrorists only accept beige foods and the ransom is your sanity.


### Token analysis

In [2]:
reponse.__dict__

{'sdk_http_response': HttpResponse(
   headers=<dict len=11>
 ),
 'candidates': [Candidate(
    content=Content(
      parts=[
        Part(
          text="""Here are 5 jokes about parenting:
  
  1.  Why do parents always look tired?
      Because "sleep like a baby" is a cruel, cruel lie.
  
  2.  My kids have selective hearing. They can hear a candy wrapper crinkle from two rooms away, but not me asking them to clean their room.
  
  3.  What's it like having a toddler?
      Imagine living with a tiny, drunk roommate who thinks glitter is a condiment and the floor is a canvas.
  
  4.  My child just discovered the word "why." Our conversations now resemble a philosophical debate between a zen master and a tiny, incredibly persistent parrot.
  
  5.  Dinner with kids is basically a culinary hostage negotiation where the terrorists only accept beige foods and the ransom is your sanity."""
        ),
      ],
      role='model'
    ),
    finish_reason=<FinishReason.STOP: 'STOP'>,
  

In [ ]:
from pydantic import BaseModel

if isinstance(reponse, BaseModel):
    metadata = reponse.usage_metadata
    print(f"Output token count = {metadata.candidates_token_count}")  # Number of tokens in model's response
    print(f"Input token count = {metadata.prompt_token_count}")  # Number of tokens in user's prompt
    print(f"Reasoning token count = {metadata.thoughts_token_count}")  # Number of tokens used for internal thinking (can be tuned/decreased)
    print(f"Total token count = {metadata.total_token_count}")  # Number of total tokens used

Output token count = 163
Input token count = 11
Reasoning token count = 585
Total token count = 759


### Thinking configuration

In [11]:
from google.genai import types

reponse_configured = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="Generate 5 jokes about parenting in markdown format.",
    config=types.GenerateContentConfig(
        thinking_config=types.ThinkingConfig(thinking_budget=0)  # Less thinking, faster output. What about quality?
    )
)

print(reponse_configured.text)
print(reponse_configured.usage_metadata)

Here are 5 jokes about parenting in markdown format:

1. My kid told me, "I love you to the moon and back!" I replied, "That's great, but can you go to your room and pick up your toys first?"

2. Parenthood is just a fancy word for "constantly asking your kids if they need to go to the bathroom."

3. What's a parent's favorite type of music? Anything quiet.

4. I used to think I was a patient person. Then I had kids. Now I know I'm just a person who tolerates a lot of noise.

5. Why did the parent bring a ladder to the playground? Because they heard the kids were going to raise the roof!
cache_tokens_details=None cached_content_token_count=None candidates_token_count=156 candidates_tokens_details=None prompt_token_count=11 prompt_tokens_details=[ModalityTokenCount(
  modality=<MediaModality.TEXT: 'TEXT'>,
  token_count=11
)] thoughts_token_count=None tool_use_prompt_token_count=None tool_use_prompt_tokens_details=None total_token_count=167 traffic_type=None


### System instruction

In [16]:
system_instruction = "You are a joking robot which always tells a joke about given topics."

prompt = "Weather."

reponse_instructed = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=prompt,
    config=types.GenerateContentConfig(
        thinking_config=types.ThinkingConfig(thinking_budget=0),
        system_instruction=system_instruction
    )
)

print(reponse_instructed.text)
print(reponse_instructed.usage_metadata)

Why did the weather report go to school? 

Because it wanted to be a little "cloudier" about its predictions! 😜
cache_tokens_details=None cached_content_token_count=None candidates_token_count=28 candidates_tokens_details=None prompt_token_count=18 prompt_tokens_details=[ModalityTokenCount(
  modality=<MediaModality.TEXT: 'TEXT'>,
  token_count=18
)] thoughts_token_count=None tool_use_prompt_token_count=None tool_use_prompt_tokens_details=None total_token_count=46 traffic_type=None


### Temperature (how creative the model is)

In [17]:
story_prompt = "Write a 2-sentence story about a panda."

story_boring = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=story_prompt,
    config=types.GenerateContentConfig(
        # thinking_config=types.ThinkingConfig(thinking_budget=0),
        # system_instruction=system_instruction
        temperature=0
    )
)

print(story_boring.text)

A fluffy panda cub, startled by a falling leaf, tumbled playfully down a mossy bank. It landed with a soft thud, then immediately began to chew on the very leaf that had surprised it.


In [18]:
story_creative = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=story_prompt,
    config=types.GenerateContentConfig(
        # thinking_config=types.ThinkingConfig(thinking_budget=0),
        # system_instruction=system_instruction
        temperature=2.0
    )
)

print(story_creative.text)

Baozi the panda found a particularly sturdy bamboo shoot and sat down with a satisfied sigh. He munched contentedly, the crisp sound filling the peaceful forest afternoon.


### Multi-modal input

In [19]:
reponse_multimodal = client.models.generate_content(
    model="gemini-2.5-flash",
    contents={
        "parts": [
            {"text": "Tell me about this place in markdown format."},
            {
                "inline_data": {
                    "mime_type": "image/png",
                    "data": open("assets/sweden.png", "rb").read()
                }
            }
        ]
    }
)

print(reponse_multimodal.text)

Here's a description of the place shown in the image, formatted in markdown:

# Stockholm, Sweden: The Beauty on Water

The image presents a stunning aerial panoramic view of **Stockholm, Sweden**, the nation's capital. It beautifully showcases the city's unique geography, sprawled across numerous islands where freshwater Lake Mälaren meets the Baltic Sea.

## Key Features Visible:

*   **Iconic Landmarks:**
    *   **Stockholm City Hall (Stockholms Stadshus):** Dominating the skyline on the right, with its distinctive golden crown atop a tall brick tower. This iconic building is renowned for hosting the Nobel Banquet.
    *   **Nordiska Museet (Nordic Museum):** In the mid-ground towards the left, a large, grand red-brick building with multiple spires is visible, likely the Nordic Museum on the island of Djurgården.
    *   **Historic Buildings:** Numerous other historic and colorful buildings line the waterfronts, showcasing diverse architectural styles ranging from medieval to turn-

In [21]:
from pathlib import Path

Path("exports").mkdir(exist_ok=True)

with open("exports/image_description.md", "w", encoding="utf-8") as f:
    f.write(reponse_multimodal.text)